In [3]:
import numpy as np
import json
import math
import copy
import pandas as pd
import dill
import os
import sys



In [2]:
BodyBrain_stats = pd.read_csv('/media/leguiart/LuisExtra/ExperimentsData2/BodyBrainQN_stats.csv')
BodyBrain_raw = pd.read_csv('/media/leguiart/LuisExtra/ExperimentsData2/BodyBrainQN.csv')

In [3]:
print(BodyBrain_stats.shape)
print(BodyBrain_raw.shape)


(2382097, 9)
(1931430, 33)


In [8]:
BodyBrain_raw.isnull().sum()

id                         0
generation                 0
run                        0
md5                        0
fitness                    0
unaligned_novelty          0
aligned_novelty            0
gene_diversity             0
control_gene_div           0
morpho_gene_div            0
morpho_div                 0
endpoint_div               0
trayectory_div             0
inipoint_x                 0
inipoint_y                 0
inipoint_z                 0
endpoint_x                 0
endpoint_y                 0
endpoint_z                 0
trayectory_x               0
trayectory_y               0
trayectory_z               0
morphology_active          0
morphology_passive         0
control_cppn_nodes         0
control_cppn_edges         0
control_cppn_ws            0
morpho_cppn_nodes          0
morpho_cppn_edges          0
morpho_cppn_ws             0
simplified_gene_div        0
simplified_gene_ne_div     0
simplified_gene_nws_div    0
dtype: int64

In [4]:
#partitioning_indexes = BodyBrain_raw.index[BodyBrain_raw['id'] == 0].tolist()
#partitioning_indexes += [len(BodyBrain_raw) - 1]
#grouped_by_run = [BodyBrain_raw.iloc[partitioning_indexes[i]:partitioning_indexes[i+1],:] for i in range(len(partitioning_indexes) - 1)]
grouped_by_run = [BodyBrain_raw[BodyBrain_raw['run'] == i] for i in range(1, 21)]

In [5]:
for i, run_df in enumerate(grouped_by_run):
    if len(run_df) > 90000:
        print(i + 1)
    #print(len(run_df))

1
2
4
6
8
10
12
14
16
18
20


In [11]:
for i, run_df in enumerate(grouped_by_run):
    if len(run_df) > 90000:
        # First remove duplicates
        df = run_df.copy()
        df = df.drop_duplicates(subset = ['id'], keep = 'last')
        # Then remove any row after generation 3000
        df = df.iloc[:90000, :]
        # And detect where the gen indexes are restarted
        maxGenSoFar = -np.inf
        gen_restart_indx = 0
        
        for j in range(len(df)):
            gen_restart_indx = j
            row = df.iloc[j, :]
            #print(row)
            if row['generation'] > maxGenSoFar:
                maxGenSoFar = row['generation']
            elif row['generation'] < maxGenSoFar:
                break
        df.iloc[gen_restart_indx:,1].loc['generation'] = np.array([gen for _ in range(30) for gen in range(maxGenSoFar + 1, 3001)])
        grouped_by_run[i] = df

In [12]:
cleaned_df_path = '/media/leguiart/LuisExtra/ExperimentsData2/BodyBrainQN_cleaned.csv'

grouped_by_run[0].to_csv(cleaned_df_path,mode = 'w', index=False)
for run_df in grouped_by_run[1:]:
    run_df.to_csv(cleaned_df_path, mode = 'a', header = False, index=False)

In [16]:
len(grouped_by_run)

20

In [57]:
stats_grouped_by_run = [BodyBrain_stats[BodyBrain_stats['Run'] == i] for i in range(1, 21)]

In [58]:
for run_df in stats_grouped_by_run:
    print(len(run_df))

127687
116698
111000
132053
111000
146150
111000
115107
111000
177156
111000
111333
111000
113294
111000
111222
111000
114293
111000
118104


In [59]:
g_list = []
for g in range(1, 3001):
    for _ in range(37):
        g_list += [g]

for i, run_df in enumerate(stats_grouped_by_run):
    if len(run_df) > 37 * 3000:
        df = run_df.copy()
        maxGenSoFar = -np.inf
        gen_restart_indx = []
        
        # We remove duplicates
        for j in range(len(df)):
            row = df.iloc[j, :]
            #print(row)
            if row['Generation'] > maxGenSoFar:
                maxGenSoFar = row['Generation']
            elif row['Generation'] < maxGenSoFar:
                gen_restart_indx += [j]
                maxGenSoFar = row['Generation']
                
        gen_restart_indx += [len(df) - 1]
        new_df = df.iloc[0:gen_restart_indx[0],:]
        
        for k in range(len(gen_restart_indx) - 1):
            asc_gen_df_slice = df.iloc[gen_restart_indx[k]:gen_restart_indx[k + 1], :]
            max_gen_reached = asc_gen_df_slice.iloc[-1,:]['Generation']
            max_df_size = 37 * max_gen_reached
            if len(asc_gen_df_slice) > max_df_size:
                rows_to_remove = len(asc_gen_df_slice) - max_df_size
                asc_gen_df_slice = asc_gen_df_slice.iloc[rows_to_remove - 1 :, :]
            new_df = new_df.append(asc_gen_df_slice, ignore_index=True)
        
        # new_df should have no duplicates now
        if len(new_df) > 37 * 3000:
            new_df = new_df.iloc[0 : 37 * 3000, :]

        new_df['Generation'] = np.array(g_list)
        #print(new_df)
        stats_grouped_by_run[i] = new_df.copy()
            

In [60]:
for run_df in stats_grouped_by_run:
    print(len(run_df))

111000
111000
111000
111000
111000
111000
111000
111000
111000
111000
111000
111000
111000
111000
111000
111000
111000
111000
111000
111000


In [61]:
cleaned_df_path = '/media/leguiart/LuisExtra/ExperimentsData2/BodyBrainQN_stats_cleaned.csv'

stats_grouped_by_run[0].to_csv(cleaned_df_path,mode = 'w', index=False)
for run_df in stats_grouped_by_run[1:]:
    run_df.to_csv(cleaned_df_path, mode = 'a', header = False, index=False)

In [3]:
import numpy as np
import json
import math
import copy
import pandas as pd
import dill
import os
import sys

BodyBrain_stats = pd.read_csv('/media/leguiart/LuisExtra/ExperimentsData2/BodyBrain_stats.csv')
BodyBrain_raw = pd.read_csv('/media/leguiart/LuisExtra/ExperimentsData2/BodyBrain.csv')
diversities = pd.read_csv('/media/leguiart/LuisExtra/ExperimentsData2/gene_div.csv')

In [7]:
so_diversities = diversities[diversities.Method == 'SO']
so_diversities

,Id,control_div,morpho_div,gene_div,Generation,Run,Method
0,28,4.839744,2.664725,5.583400,1,1,SO
1,7,5.995350,3.413436,6.998795,1,1,SO
2,19,5.434079,3.229738,6.393694,1,1,SO
3,23,4.140472,3.219485,5.306388,1,1,SO
4,4,4.988583,3.521787,6.181821,1,1,SO
...,...,...,...,...,...,...,...
1799695,89959,0.938223,0.230222,0.969852,3000,20,SO
1799696,89800,1.087298,0.159899,1.114267,3000,20,SO
1799697,89895,0.790429,0.262088,0.855560,3000,20,SO
1799698,89955,0.984558,0.168591,1.009124,3000,20,SO


In [8]:
so_divs_grouped_by_run = [so_diversities[so_diversities['Run'] == i] for i in range(1, 21)]
so_info_grouped_by_run = [BodyBrain_raw[BodyBrain_raw['run'] == i] for i in range(1, 21)]

In [9]:
for info in so_divs_grouped_by_run:
    print(len(info))

NameError: name 'so_divs_grouped_by_run' is not defined

In [15]:
parent_pop_dfs = []
for i, run_df in enumerate(so_divs_grouped_by_run):
    parent_pop_dfs += [run_df.join(so_info_grouped_by_run[i].set_index('id'), on='Id', lsuffix='_caller', rsuffix='_other')]

In [16]:
parents_df_path = '/media/leguiart/LuisExtra/ExperimentsData2/BodyBrainParents.csv'

parent_pop_dfs[0].to_csv(parents_df_path,mode = 'w', index=False)
for run_df in parent_pop_dfs[1:]:
    run_df.to_csv(parents_df_path, mode = 'a', header = False, index=False)

In [5]:
original_indicator_csv = pd.read_csv('/media/leguiart/LuisExtra/ExperimentsData2/BodyBrainQN.csv')
repeated_indicator_csv = pd.read_csv('/media/leguiart/LuisExtra/ExperimentsData2/BodyBrainQN2.csv')

In [10]:
original_indicator_csv.head(10)


,id,generation,run,md5,fitness,unaligned_novelty,aligned_novelty,gene_diversity,control_gene_div,morpho_gene_div,...,morphology_passive,control_cppn_nodes,control_cppn_edges,control_cppn_ws,morpho_cppn_nodes,morpho_cppn_edges,morpho_cppn_ws,simplified_gene_div,simplified_gene_ne_div,simplified_gene_nws_div
0,14,1,1,30bc4d0e040515a8e4ba5c1c11dae71a,0.145117,12.364630,0.120268,6.817174,5.983003,3.042246,...,26,10,17,4.470540,12,20,-0.581597,6.126992,5.158327,3.314432
1,22,1,1,66eeb19b8f629d2a7d7d457194f190d9,0.052411,17.571045,0.035930,5.693497,4.604187,3.222912,...,39,12,20,2.529017,11,18,-1.711305,5.919246,4.844229,3.173463
2,24,1,1,1116208b50bbf3af6fa8b2b4d8dd453b,0.014247,17.779497,0.012331,6.185345,4.969576,3.612268,...,26,11,19,2.641000,12,20,-2.583367,6.067610,5.323827,2.327004
3,29,1,1,a285a9163cdec824bf819599eb526752,0.000580,27.924151,0.004112,6.349990,5.212136,3.486593,...,79,12,20,-1.584652,12,20,-0.543292,5.786691,4.895850,2.273843
4,7,1,1,b76704ae5d7956f2e7c019fdab617089,0.090576,12.182152,0.054919,5.286071,3.777965,3.599394,...,49,11,18,-3.552462,11,18,0.843492,6.560180,5.775205,3.037440
5,0,1,1,2f5e6549d406f0d2b1989d4919a85db3,0.041018,12.620491,0.021381,5.333360,3.903149,3.508624,...,17,12,20,1.791111,12,20,1.203642,4.732056,3.773725,2.273843
6,10,1,1,edb53189ff7717e9a304ec0874882f29,0.046409,10.948968,0.031220,5.323972,4.373397,2.943448,...,8,11,19,-1.372315,12,20,2.117637,5.174018,4.375058,2.327004
7,27,1,1,100a306e2c716d2398b975c40374899a,0.043919,17.107258,0.037834,5.188458,4.115925,3.059458,...,47,11,18,0.503681,11,18,-0.776788,5.255524,4.189742,3.037440
8,19,1,1,597b31d5b691738635c318e7ec8551e4,0.003732,13.315251,0.006440,5.648030,4.585419,3.184659,...,6,9,13,1.402998,12,20,1.081845,7.312010,4.262914,6.223011
9,3,1,1,9318f8fd880985e9c6d4668be397d44d,0.000000,22.838990,0.003856,5.440956,4.508648,2.903559,...,15,12,20,5.553271,12,20,7.136739,8.387798,7.829881,2.273843


In [9]:
repeated_indicator_csv.head(10)
repeated_indicator_csv.isna().sum()

id                         0
generation                 0
run                        0
md5                        0
fitness                    0
unaligned_novelty          0
aligned_novelty            0
gene_diversity             0
control_gene_div           0
morpho_gene_div            0
morpho_div                 0
endpoint_div               0
trayectory_div             0
inipoint_x                 0
inipoint_y                 0
inipoint_z                 0
endpoint_x                 0
endpoint_y                 0
endpoint_z                 0
trayectory_x               0
trayectory_y               0
trayectory_z               0
morphology_active          0
morphology_passive         0
control_cppn_nodes         0
control_cppn_edges         0
control_cppn_ws            0
morpho_cppn_nodes          0
morpho_cppn_edges          0
morpho_cppn_ws             0
simplified_gene_div        0
simplified_gene_ne_div     0
simplified_gene_nws_div    0
dtype: int64

In [13]:
grouped_by_run = [original_indicator_csv[original_indicator_csv['run'] == i] for i in range(1, 21)]

In [14]:
for i, run_df in enumerate(grouped_by_run):
    if len(run_df) > 90000:
        grouped_by_run[i] = repeated_indicator_csv[repeated_indicator_csv['run'] == i + 1].copy()

In [15]:
cleaned_df_path = '/media/leguiart/LuisExtra/ExperimentsData2/BodyBrainQN_cleaned.csv'
grouped_by_run[0].to_csv(cleaned_df_path,mode = 'w', index=False)
for run_df in grouped_by_run[1:]:
    run_df.to_csv(cleaned_df_path, mode = 'a', header = False, index=False)

In [2]:
import numpy as np
import json
import math
import copy
import pandas as pd

In [3]:
original_indicator_csv = pd.read_csv('/media/leguiart/LuisExtra/ExperimentsData2/BodyBrainQN.csv')

In [4]:
grouped_by_run = [original_indicator_csv[original_indicator_csv['run'] == i] for i in range(1, 21)]

In [5]:
for run_df in grouped_by_run:
    print(len(run_df))

90000
90000
90000
90000
90000
90000
90000
90000
90000
90000
90000
90000
90000
90000
90000
90000
90000
90000
90000
90000


In [6]:
repeated_indicator_csv = pd.read_csv('/media/leguiart/LuisExtra/ExperimentsData2/BodyBrainQN2.csv')

In [8]:
len(repeated_indicator_csv[repeated_indicator_csv['run'] == 2])

90000